## CAMD Meta agent example

In [1]:
from camd.campaigns.meta_agent import MetaAgentCampaign, \
    StabilityCampaignAnalyzer
from camd.analysis import StabilityAnalyzer
from camd.utils.data import load_dataframe, partition_intercomp, \
    get_oqmd_data_by_chemsys
from camd.experiment.agent_simulation import LocalAgentSimulation
from camd.agent.base import RandomAgent
from taburu.table import ParameterTable
from camd import CAMD_S3_BUCKET
# Note that env variable must be set
if CAMD_S3_BUCKET is None:
    raise ValueError("Set CAMD_S3_BUCKET to enable notebook")

### Define the agents and agent_pool

In [2]:
# Define agent parameters
agent_params = [
    {
        "@class": ["camd.agent.base.RandomAgent"],
        "n_query": [2, 3],
    },
]

### Construct the dataframe using Fe-O data

In [3]:
data = get_oqmd_data_by_chemsys("Fe-O", drop_duplicates=True)
candidate_data, seed_data = partition_intercomp(data)

### Construct core campaign objects

In [4]:
experiment = LocalAgentSimulation(
    atf_candidate_data=candidate_data, seed_data=seed_data,
    analyzer=StabilityAnalyzer(), iterations=10,
)
analyzer = StabilityCampaignAnalyzer(
    checkpoint_indices=[2, 5, 10])
agent_pool = ParameterTable(agent_params)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 4092.00it/s]


### Reserve campaign in S3

In [5]:
MetaAgentCampaign.reserve(
    name="test_meta_agent_python", experiment=experiment,
    agent_pool=agent_pool, analyzer=analyzer
)

### Run via Python API

In [6]:
import os, shutil
shutil.rmtree("python", ignore_errors=True)
os.makedirs("python")
os.chdir("python")
meta_agent = RandomAgent(n_query=1)
campaign = MetaAgentCampaign.from_reserved_name(
    "test_meta_agent_python", meta_agent=meta_agent)
campaign.autorun()
os.chdir('..')

MetaAgentCampaign initialization state: Agent RandomAgent hypothesizing
MetaAgentCampaign 0 state: Running experiments
Iteration: 0
MetaAgentCampaign 0 state: Getting new results
Campaign initialization state: Agent RandomAgent hypothesizing
Campaign 0 state: Running experiments
Iteration: 0
Campaign 0 state: Getting new results
Campaign 0 state: Analyzing results
/Users/josephmontoya/miniconda3/envs/camd/lib/python3.7/site-packages/pymatgen/analysis/phase_diagram.py:1968: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "ko-" (-> linestyle='-'). The keyword argument will take precedence.
  plt.plot(x, y, "ko-", **self.plotkwargs)
Campaign 0 state: Agent RandomAgent hypothesizing
Campaign 0 state: Running experiments
  Waiting for next round ...
Iteration: 1
Campaign 1 state: Getting new results
Campaign 1 state: Analyzing results
Campaign 1 state: Agent RandomAgent hypothesizing
Campaign 1 state: Running experiments
  Waiting for nex

PicklingError: logger cannot be pickled

### Run via CLI

In [ ]:
def create_test(name):
    data = get_oqmd_data_by_chemsys("Fe-O", drop_duplicates=True)
    candidate_data, seed_data = partition_intercomp(data)
    experiment = LocalAgentSimulation(
        atf_candidate_data=candidate_data, seed_data=seed_data,
        analyzer=StabilityAnalyzer(), iterations=10,
    )
    analyzer = StabilityCampaignAnalyzer(
        checkpoint_indices=[2, 5, 10])
    agent_pool = ParameterTable(agent_params)
    MetaAgentCampaign.reserve(
        name=name, experiment=experiment,
        agent_pool=agent_pool, analyzer=analyzer
    )

In [ ]:
create_test("test_meta_agent_cli")

In [ ]:
# Run s3 backed campaign
!rm -rf test && \
mkdir test && \
cd test && \
camd_runner meta_agent/test_meta_agent_cli

Summary figure
![title](test/campaign_summary.png)

### Run on AWS Batch

In [ ]:
create_test("test_meta_agent_aws_batch")

In [ ]:
submit_output = \
!aws batch submit-job \
--job-name test_meta_agent_aws_batch \
--job-queue camd_dev \
--job-definition camd_dev_batch \
--parameters name=meta_agent/test_meta_agent_aws_batch

In [ ]:
import json, time
job_data = json.loads(''.join(submit_output))
job_id = job_data['jobId']
while True:
    describe_output = !aws batch describe-jobs --jobs $job_id
    job_data = json.loads(''.join(describe_output))
    job_status = job_data['jobs'][0]['status']
    print("Job status is {}".format(job_status))
    if job_status not in ['PENDING', 'RUNNABLE', 'RUNNING']:
        break
    time.sleep(20)